In [254]:
# adding noWay & isClosed
# Imports
import networkx as nx
import numpy as np
import cvxpy as cp

In [255]:
# Creating the test graph
G = nx.MultiDiGraph()
# G.add_edge(1, 2, weight=1, isNew=False)
G.add_edge(1, 2, weight=3, noWay=0, isClosed=0, length=20, speed=18, maxSpeed=20)
G.add_edge(1, 3, weight=5, noWay=0, isClosed=0, length=20, speed=15, maxSpeed=20)
G.add_edge(2, 4, weight=1, noWay=0, isClosed=0, length=20, speed=10, maxSpeed=20)
G.add_edge(3, 4, weight=5, noWay=0, isClosed=0, length=20, speed=7, maxSpeed=20)
G.add_edge(4, 5, weight=5, noWay=0, isClosed=1, length=20, speed=5, maxSpeed=20)

#list(G.edges(data=True))
desiredPath = [1, 3, 4]

In [256]:
# def addMissingEdges(graph):
#     newGraph = graph.copy()
    
#     return newGraph

In [257]:
def getInverseSpeed(speed):
    return 1/speed
def getSpeedFromInverse(inverse):
    return 1/inverse

In [258]:
#TODO: remove the weights from the edges, and calculate them using the variables.
    # Change the objective, to ||v1' - v1|| + || v2' - v2|| + .... and so on.

def inverseShortestPath(original_graph, desiredPath):
    # auxiliary variables
    
    graph = original_graph.copy()

    inf = 1000000
        
    n = len(graph.nodes())

    m = len(graph.edges())
    
    source = desiredPath[0]
    target = desiredPath[len(desiredPath) - 1]

    weights = []
    noWay = []
    areClosed = []
    inverseSpeeds = []
    inverseMaxSpeeds = []
    lengths = []

    edges = []
    edgeIndex = {}
    for (i, j, data) in graph.edges(data=True):
        # Edges Index
        edgeIndex[i,j] = len(edges)
        # Add edge
        edges.append([i, j])
        
        # Data:
        # Edge Weight
        weights.append(data['weight'])
        # (not) is there an edge:
        noWay.append(data['noWay'])
        # closed or not
        areClosed.append(data['isClosed'])
        #get the inverse of the speed
        inverseSpeeds.append(getInverseSpeed(data['speed']))
        #get the inverse of the max speed
        inverseMaxSpeeds.append(getInverseSpeed(data['maxSpeed']))
        
        lengths.append(getInverseSpeed(data['length']))
        
        
    # Add inverse edges with infinity weight
    for (i, j) in graph.edges():
        if (j, i) not in graph.edges():
            # Edges Index
            edgeIndex[j,i] = len(edges)
            # Add edge
            edges.append([j, i])
            
            # Data:
            # Edge Weight
            weights.append(inf)
            # (not) is there an edge:
            noWay.append(1)
            # closed or not
            areClosed.append(0)
            
            inverseSpeeds.append(1)
            inverseMaxSpeeds.append(inf)
            
            lengths.append(1)
            
            #graph.add_edge(j, i, weight=inf, noWay=1)
            graph.add_edge(j, i, weight=1, noWay=1, isClosed=0, lenngth=1, speed=1, maxSpeed=inf)
        
        
    nodeIndex = {}
    nodes = []
    for n in graph.nodes:
        # Node Index
        nodeIndex[n] = len(nodes)
        # Add Node
        nodes.append(n)
        
        
    # Ax = b
    A = np.zeros([len(nodes), len(edges)])
    b = np.zeros(len(nodes))

    for i in range(len(nodes)):
        for neighbour in graph.adj[nodes[i]]:
            # Filling A
            j = edgeIndex[nodes[i], neighbour]
            A[i,j] = 1
            j = edgeIndex[neighbour, nodes[i]]
            A[i,j] =-1
            
        # Filling b
        if nodes[i] == source:
            b[i] = 1
        if nodes[i] == target:
            b[i] = -1
            
            
    # optimal x
    path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    xstar = np.zeros(len(edges))
    for p in range(len(path)-1):
        j = edgeIndex[path[p], path[p+1]]
        xstar[j] = 1

    # desired x
    xzero = np.zeros(len(edges))
    for p in range(len(desiredPath)-1):
        j = edgeIndex[desiredPath[p], desiredPath[p+1]]
        xzero[j] = 1
        


    # LP (ISP):
    
    w_original = np.asarray(weights)
    noWay_original = np.asarray(noWay)
    areClosed_original = np.asarray(areClosed)
    inverseSpeeds_original = np.asarray(inverseSpeeds)
    inverseMaxSpeeds_original = np.asarray(inverseMaxSpeeds)
    
    w_ = cp.Variable(len(w_original), integer=True)
    pi_ = cp.Variable(len(nodes)) #(2d)
    lambda_ = cp.Variable(len(edges)) #(2d)
    
    # more vars
    noWay_ = cp.Variable(len(noWay_original), boolean=True)
    areClosed_ = cp.Variable(len(areClosed_original), boolean=True)
    inverseSpeeds_ = cp.Variable(len(inverseSpeeds_original))
    inverseMaxSpeeds_ = cp.Variable(len(inverseMaxSpeeds_original))
    
    # weights for L1norm (not-on-path-penalty) # Why do we use this???????
    # penalty = np.array([1]*len(w_original))
    # for i in range(len(nodes)):
    #     if nodes[i] not in desiredPath[1:-1]:
    #         penalty[i] = 10
        
        
        
    constraints = []
    
    # (2b) & (2c) 
    for j in range(len(edges)):
        d_j = w_[j] + inf * noWay_[j] + inf * areClosed_[j]
        #d_j = inverseSpeeds_[j] * lengths[j]  + inf * noWay_[j] + inf * areClosed_[j]
        
        if xzero[j] == 1:
        # sum_i a_ij * pi_i = d_j,              for all j in desired path (2b)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) == d_j )
        else:
        # sum_i a_ij * pi_i + lambda_j = d_j,   for all j not in desired path (2c)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) + lambda_[j] == d_j )
        
        
    #(2e)
    for j in range(len(edges)):
        if xzero[j] == 0:
            constraints.append( lambda_[j] >= 0 )
            
            
    # sum_k l_ik = 1, for all i (5g, from NISP)
    # for i in range(len(varnodes)):
    #     idx = len(allowedAreaTypes) * i
    #     constraints.append( cp.sum(l_[idx:idx+len(allowedAreaTypes)]) == 1 )
            
    epsilon = 0.001
    # w>=0
    constraints.append(w_ >= epsilon)
    
    for j in range(len(edges)):
        if xzero[j] == 0:
            #if not in desired path, do not change the data 
            constraints.append( w_[j] == w_original[j] )
            constraints.append( noWay_[j] == noWay_original[j] )
            constraints.append( areClosed_[j] == areClosed_original[j] )
            constraints.append( inverseSpeeds_[j] == inverseSpeeds_original[j] )
            constraints.append( inverseMaxSpeeds_[j] == inverseMaxSpeeds_original[j] )
        
        
        # inverseSpeed is at least the max speed; ie speed is at most max speed.
        else:
            constraints.append(  inverseSpeeds_[j] >= inverseMaxSpeeds_original[j] )
    
            
            
    
    # # new edges:
    # for j in range(len(edges)):
    #     d_j = w_[j]
    #     if noWay_original[j] == 0:
    #         #if not in desired path, do not change the weight 
    #         constraints.append( w_[j] == w_original[j] )
    
    
    
    
    
    
    cost1 = cp.norm1(w_ - w_original)
    cost2 = cp.norm1(noWay_ - noWay_original)
    cost3 = cp.norm1(areClosed_ - areClosed_original)
            
            
    # Cost funnction
    cost = cost1 + cost2 + cost3 # ||w' - w||1 (2a)
    
    #Check why the penalty??
    #cost = cp.norm1(cp.multiply(w_ - w_original, penalty))
        
    # Forming the problem
    prob = cp.Problem(cp.Minimize(cost), constraints)
    
    # Solve the problem
    #prob.solve(verbose=True)#Detailed
    prob.solve()  # Returns the optimal value.
    print("\nThe optimal value is", prob.value)
    print("A solution w' is")
    print(w_.value)
    print('original isClosed: ', areClosed_original)
    print('new isClosed: ', areClosed_.value)
    
    
    newGraph = nx.MultiDiGraph()
    
    for (i, j), index in edgeIndex.items():
        newGraph.add_edge(i, j, weight = w_.value[index], noWay = noWay_.value[index], isClosed = areClosed_.value[index])
        
    return newGraph


In [259]:
# Solving using the function

new_graph = inverseShortestPath(G, desiredPath)


The optimal value is 6.0
A solution w' is
[3.e+00 3.e+00 1.e+00 1.e+00 5.e+00 1.e+06 1.e+06 1.e+06 1.e+06 1.e+06]
original isClosed:  [0 0 0 0 1 0 0 0 0 0]
new isClosed:  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [260]:
nx.shortest_path(new_graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    

[1, 2, 4]

In [261]:
desiredPath

[1, 3, 4]

In [262]:
G.edges(data=True)

OutMultiEdgeDataView([(1, 2, {'weight': 3, 'noWay': 0, 'isClosed': 0, 'length': 20, 'speed': 18, 'maxSpeed': 20}), (1, 3, {'weight': 5, 'noWay': 0, 'isClosed': 0, 'length': 20, 'speed': 15, 'maxSpeed': 20}), (2, 4, {'weight': 1, 'noWay': 0, 'isClosed': 0, 'length': 20, 'speed': 10, 'maxSpeed': 20}), (3, 4, {'weight': 5, 'noWay': 0, 'isClosed': 0, 'length': 20, 'speed': 7, 'maxSpeed': 20}), (4, 5, {'weight': 5, 'noWay': 0, 'isClosed': 1, 'length': 20, 'speed': 5, 'maxSpeed': 20})])

In [263]:
new_graph.edges(data=True)

OutMultiEdgeDataView([(1, 2, {'weight': 3.0, 'noWay': 0.0, 'isClosed': 0.0}), (1, 3, {'weight': 3.0, 'noWay': 0.0, 'isClosed': 0.0}), (2, 4, {'weight': 1.0, 'noWay': 0.0, 'isClosed': 0.0}), (2, 1, {'weight': 1000000.0, 'noWay': 1.0, 'isClosed': 0.0}), (3, 4, {'weight': 1.0, 'noWay': 0.0, 'isClosed': 0.0}), (3, 1, {'weight': 1000000.0, 'noWay': 1.0, 'isClosed': 0.0}), (4, 5, {'weight': 5.0, 'noWay': 0.0, 'isClosed': 1.0}), (4, 2, {'weight': 1000000.0, 'noWay': 1.0, 'isClosed': 0.0}), (4, 3, {'weight': 1000000.0, 'noWay': 1.0, 'isClosed': 0.0}), (5, 4, {'weight': 1000000.0, 'noWay': 1.0, 'isClosed': 0.0})])